# Interventions

This tutorial shows how to add testing and treatment interventions to an STIsim simulation. We'll start with a simple gonorrhea example, then show how HIV interventions work.

## Simple STI testing and treatment

The most common pattern is: create a treatment, create a test that triggers treatment for positive results, and pass both to the sim.

Let's add routine testing and treatment for gonorrhea:

In [ ]:
import stisim as sti
import starsim as ss
import pandas as pd
import numpy as np

# Create the disease
ng = sti.Gonorrhea(init_prev=0.05)

# Create a treatment: 90% efficacy by default
ng_tx = sti.STITreatment(diseases='ng', name='ng_tx')

# Create syndromic management: symptomatic people seek care and get treated
syndromic = sti.SymptomaticTesting(
    diseases=[ng],
    treatments=[ng_tx],
    disease_treatment_map={'ng': ng_tx},
)

# Run with and without treatment
sim_base = sti.Sim(diseases=ng, n_agents=2000, start=2010, stop=2030)

ng2 = sti.Gonorrhea(init_prev=0.05)
ng_tx2 = sti.STITreatment(diseases='ng', name='ng_tx')
syndromic2 = sti.SymptomaticTesting(
    diseases=[ng2],
    treatments=[ng_tx2],
    disease_treatment_map={'ng': ng_tx2},
)
sim_intv = sti.Sim(diseases=ng2, interventions=[syndromic2, ng_tx2], n_agents=2000, start=2010, stop=2030)

sim_base.run(verbose=0)
sim_intv.run(verbose=0)

In [ ]:
import matplotlib.pyplot as plt

fig, ax = plt.subplots()
ax.plot(sim_base.timevec, sim_base.results.ng.prevalence, label='No treatment')
ax.plot(sim_intv.timevec, sim_intv.results.ng.prevalence, label='Syndromic management')
ax.set_xlabel('Year')
ax.set_ylabel('NG prevalence')
ax.set_title('Effect of syndromic management on gonorrhea')
ax.legend()
fig

## HIV testing and ART

HIV interventions follow the same pattern but use HIV-specific classes. `HIVTest` tests undiagnosed agents, and `ART` puts diagnosed agents on treatment with coverage targets over time.

In [ ]:
# Create HIV testing: 10% annual probability, starting in 2000
hiv_test = sti.HIVTest(test_prob_data=0.1, start=2000, name='hiv_test')

# Create ART: coverage ramps from 0% to 80% over time
art_data = pd.DataFrame(
    index=np.arange(2000, 2031),
    data={'p_art': np.clip(np.linspace(0, 0.9, 31), 0, 1)}
)
art = sti.ART(coverage_data=art_data)

# Run with and without interventions
sim_base = sti.Sim(diseases='hiv', n_agents=3000, start=2000, stop=2030)
sim_intv = sti.Sim(diseases='hiv', interventions=[hiv_test, art], n_agents=3000, start=2000, stop=2030)

sim_base.run(verbose=0)
sim_intv.run(verbose=0)

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(10, 4))

axes[0].plot(sim_base.timevec, sim_base.results.hiv.new_infections, label='No ART', alpha=0.7)
axes[0].plot(sim_intv.timevec, sim_intv.results.hiv.new_infections, label='Testing + ART', alpha=0.7)
axes[0].set_xlabel('Year')
axes[0].set_ylabel('New HIV infections')
axes[0].legend()

axes[1].plot(sim_intv.timevec, sim_intv.results.hiv.p_on_art)
axes[1].set_xlabel('Year')
axes[1].set_ylabel('Proportion on ART')
axes[1].set_title('ART coverage')

plt.tight_layout()
fig

## Targeting interventions

Use the `eligibility` parameter to target interventions to specific populations. For example, FSW-targeted testing:

In [ ]:
# Higher testing rate for FSWs
fsw_test = sti.HIVTest(
    test_prob_data=0.3,
    start=2000,
    name='fsw_test',
    eligibility=lambda sim: sim.networks.structuredsexual.fsw,
)

# Lower testing rate for the general population
gp_test = sti.HIVTest(
    test_prob_data=0.05,
    start=2000,
    name='gp_test',
    eligibility=lambda sim: ~sim.networks.structuredsexual.fsw,
)

art = sti.ART(coverage_data=art_data)

sim = sti.Sim(
    diseases='hiv',
    interventions=[fsw_test, gp_test, art],
    n_agents=3000,
    start=2000,
    stop=2030,
)
sim.run(verbose=0)
sim.plot(key=['hiv.prevalence', 'hiv.new_infections', 'hiv.p_on_art'])

## Exercises

1. Add VMMC to the HIV sim using `sti.VMMC()`. How does it affect new infections compared to ART alone?
2. Modify the gonorrhea example to use a higher treatment efficacy (`treat_eff=0.99`). How much does prevalence change?
3. Create a sim with both gonorrhea and chlamydia, and use a single `STITreatment(diseases=['ng', 'ct'])` to treat both. Compare prevalence with and without treatment.